# Azure PowerShell VM Visualization demo
> A Notebook mixing Azure PowerShell and Plotly

## Prerequisites

### Install required modules

You'll need to install `Az.Compute` and `Az.Resources` for this Notebook.

In [1]:
Install-Module Az.Compute,Az.Resources -Force

### Authenticate and set context

First connect to your Azure account.

In [1]:
Connect-AzAccount


Account                SubscriptionName TenantId                             Environment
-------                ---------------- --------                             -----------
tyleonha@microsoft.com Thanks Microsoft 72f988bf-86f1-41af-91ab-2d7cd011db47 AzureCloud



**(Optional)** If your account contains more than one active subscription the first one will be selected for further use. To select another subscription, use Set-AzContext.

In [ ]:
Set-AzContext -Subscription "My Subscription"

### (Optional) Populate Azure with test VMs

Here's a script that will create a number of VMs. Feel free to change the value at the top.

> NOTE: This can take a LONG time.

In [2]:
# IMPORTANT VARIABLES
$RESOURCE_GROUP_NAME = 'VMVisDemo'
$LOCATION = 'East US 2'
$NUM_OF_VMs = 10
$USER_CREDENTIAL = Get-Credential


PowerShell credential request
Enter your credentials.


User:  azureuser
Password for user azureuser:  ·····················


In [4]:
Write-Host "Creting resource group."
New-AzResourceGroup -Name $RESOURCE_GROUP_NAME -Location $LOCATION | Out-Null

Write-Host "Creting VMs."
$jobs = 1..$NUM_OF_VMs | ForEach-Object {
    $splat = @{
        Image = "UbuntuLTS"
        Location = $LOCATION
        Name = "MyVM-$_"
        ResourceGroupName = $RESOURCE_GROUP_NAME
        Credential = $USER_CREDENTIAL
        AsJob = $true
    }
    New-AzVM @splat
}

# Wait for them to all be created
Wait-Job $jobs | Out-Null
"Done!"

Creting resource group.
Creting VMs.
Done!


Now we need to randomly stop a number of them so that the graph below has some variance.

In [5]:
$numOfVMsToStop = Get-Random -Minimum 2 -Maximum $NUM_OF_VMs
Write-Host "Randomly stoping $numOfVMsToStop VMs."

$vms = Get-AzVM -ResourceGroupName $RESOURCE_GROUP_NAME
$jobs = for ($i = 0; $i -lt $numOfVMsToStop; $i++) {
    $vms | Get-Random | Stop-AzVM -Force -AsJob
}

Wait-Job $jobs | Out-Null

Write-Host "Done!"

Randomly stoping 4 VMs.
Done!


**Cleanup** - If you wanna clean up these test VMs. Run this:

> NOTE: This can take a LONG time.

In [ ]:
Write-Host "Deleting VMs."
$jobs = Get-AzVM -ResourceGroupName $RESOURCE_GROUP_NAME | Remove-AzVM -AsJob -Force
Wait-Job $jobs | Out-Null

Write-Host "Deleting resource group."
Remove-AzResourceGroup -ResourceGroupName $RESOURCE_GROUP_NAME -Force | Out-Null

Write-Host "Done!"

## Demo

We'll be fetching the "PowerState" of each of our VMs in Azure and plotting it on a pie chart.

First, let's get the data:

In [6]:
$vms = Get-AzVM -Status
$data = $vms.PowerState

$groupedData = $data | Group-Object
$groupedData


Count Name                      Group
----- ----                      -----
    7 VM deallocated            {VM deallocated, VM deallocated, VM deallocated, VM deallocated…}
   10 VM running                {VM running, VM running, VM running, VM running…}



Now we can render that data into a Pie graph and plot it in a chart:

In [7]:
$trace = [Graph.Pie]@{
    name = "VM PowerState"
    labels = $groupedData.Name
    values = [int[]]($groupedData | % Count)
}

New-PlotlyChart -Title "VM Status" -Trace $trace | Out-Display